In [1]:
import sys
sys.path.append('../')

In [2]:
import torch
import yaml
from torchvision import transforms, datasets
import torchvision
import numpy as np
import pandas as pd
import os
from sklearn import preprocessing
from torch.utils.data.dataloader import DataLoader
from data.dataset import ImageDataset, FineTuneImageDataset


In [10]:
from models.resnet_base_network import ResNet18

In [3]:
batch_size = 512
data_transforms = torchvision.transforms.Compose([transforms.ToTensor()])

training_data_transform = transforms.Compose([
        #T.ToPILImage("RGB"),
        #T.RandomRotation(5),
        transforms.RandomHorizontalFlip(0.5),
        # SquarePad(),
        transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]),
    ])

test_data_transform = transforms.Compose([
        #T.ToPILImage("RGB"),
        # SquarePad(),
        transforms.Resize((128,128)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406],
                    std=[0.229, 0.224, 0.225]),
    ])

In [4]:
config = yaml.load(open("../config/config.yaml", "r"), Loader=yaml.FullLoader)

In [5]:
dataset_dir = "/home/ubuntu/BYOL/datasets/in_domain_dataset"

train_imgs_dir = os.path.join(dataset_dir, "train")
train_labels = pd.read_csv(os.path.join(dataset_dir, "label/train_label.csv"))

val_imgs_dir = os.path.join(dataset_dir, "val")
val_labels = pd.read_csv(os.path.join(dataset_dir, "label/val_label.csv"))

test_imgs_dir = os.path.join(dataset_dir, "test")
test_labels = pd.read_csv(os.path.join(dataset_dir, "label/test_label.csv"))

In [6]:
train_dataset = FineTuneImageDataset(train_labels, train_imgs_dir, transform=training_data_transform)
val_dataset = FineTuneImageDataset(val_labels, val_imgs_dir, transform=training_data_transform)
test_dataset = FineTuneImageDataset(test_labels, test_imgs_dir, transform=test_data_transform)

# test_dataset = datasets.STL10('/home/thalles/Downloads/', split='test', download=False,
#                                transform=data_transforms)


In [7]:
print("Input shape:", train_dataset[0][0].shape)

Input shape: torch.Size([3, 128, 128])


In [8]:
train_loader = DataLoader(train_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

val_loader = DataLoader(val_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)


test_loader = DataLoader(test_dataset, batch_size=batch_size,
                          num_workers=0, drop_last=False, shuffle=True)

In [11]:
device = 'cpu' #'cuda' if torch.cuda.is_available() else 'cpu'
encoder = ResNet18(**config['network'])
# encoder = torchvision.models.resnet18(pretrained=False)
output_feature_dim = encoder.projetion.net[0].in_features
# encoder.to(device)

In [12]:
#load pre-trained parameters
load_params = torch.load(os.path.join('/home/ubuntu/BYOL/runs/pretrained_resnets/checkpoints/model_3.pth'),
                         map_location=torch.device(torch.device(device)))

if 'online_network_state_dict' in load_params:
    encoder.load_state_dict(load_params['online_network_state_dict'])
    print("Parameters successfully loaded.")

output_dim = 7
# remove the projection head
# encoder = torch.nn.Sequential(*list(encoder.encoder.children()))   
encoder = encoder.encoder
# finetuner = torchvision.models.resnet18(pretrained=False)
# finetuner = finetuner.load_state_dict()
# encoder.fc = Linear(512, 7)
# encoder = encoder.to(device)

Parameters successfully loaded.


In [13]:
encoder

Sequential(
  (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (2): ReLU(inplace=True)
  (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (4): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Con

In [14]:
class LogisticRegression(torch.nn.Module):
    def __init__(self, input_dim, output_dim):
        super(LogisticRegression, self).__init__()
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        return self.linear(x)

In [15]:
class FineTuningModel(torch.nn.Module):
    def __init__(self, encoder, input_dim, output_dim):
        super(FineTuningModel, self).__init__()
        self.encoder = encoder
        self.linear = torch.nn.Linear(input_dim, output_dim)
        
    def forward(self, x):
        out = self.encoder(x)
        out =  torch.mean(out, dim=[2, 3])
        return self.linear(out)

In [121]:
logreg = LogisticRegression(output_feature_dim, 7)
logreg = logreg.to(device)

In [16]:
finetuner = FineTuningModel(encoder, output_feature_dim, 7)
finetuner = finetuner.to(device)

In [28]:
def get_features_from_encoder(encoder, loader):
    
    x_train = []
    y_train = []

    # get the features from the pre-trained model
    for i, (x, y) in enumerate(loader):
        with torch.no_grad():
            feature_vector = encoder(x)
            x_train.extend(feature_vector)
            y_train.extend(y.numpy())

            
    x_train = torch.stack(x_train)
    y_train = torch.tensor(y_train)
    return x_train, y_train

In [29]:
encoder.eval()
x_train, y_train = get_features_from_encoder(encoder, train_loader)
x_test, y_test = get_features_from_encoder(encoder, test_loader)

if len(x_train.shape) > 2:
    x_train = torch.mean(x_train, dim=[2, 3])
    x_test = torch.mean(x_test, dim=[2, 3])
    
print("Training data shape:", x_train.shape, y_train.shape)
print("Testing data shape:", x_test.shape, y_test.shape)

Training data shape: torch.Size([10738, 512]) torch.Size([10738])
Testing data shape: torch.Size([3068, 512]) torch.Size([3068])


In [30]:
def create_data_loaders_from_arrays(X_train, y_train, X_test, y_test):

    train = torch.utils.data.TensorDataset(X_train, y_train)
    train_loader = torch.utils.data.DataLoader(train, batch_size=64, shuffle=True)

    test = torch.utils.data.TensorDataset(X_test, y_test)
    test_loader = torch.utils.data.DataLoader(test, batch_size=512, shuffle=False)
    return train_loader, test_loader

In [31]:
scaler = preprocessing.StandardScaler()
scaler.fit(x_train)
x_train = scaler.transform(x_train).astype(np.float32)
x_test = scaler.transform(x_test).astype(np.float32)

In [32]:
train_loader, test_loader = create_data_loaders_from_arrays(torch.from_numpy(x_train), y_train, torch.from_numpy(x_test), y_test)

In [160]:
# optimizer = torch.optim.Adam(logreg.parameters(), lr=3e-4)
optimizer = torch.optim.Adam(finetuner.parameters(), lr=3e-4)
criterion = torch.nn.CrossEntropyLoss()
eval_every_n_epochs = 5

finetuner.train()

for epoch in range(20):
#     train_acc = []
    for x, y in train_loader:

        x = x.to(device)
        y = y.to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()        
        
#         logits = logreg(x)
        logits = finetuner(x)
        predictions = torch.argmax(logits, dim=1)
        
#         print("here")
        loss = criterion(logits, y)
        
        loss.backward()
        optimizer.step()
        print(f'Current Loss:{loss}')
        
    total = 0
    if epoch % eval_every_n_epochs == 0:
        correct = 0
        for x, y in val_loader:
            x = x.to(device)
            y = y.to(device)

#             logits = logreg(x)
            logits = finetuner(x)
            predictions = torch.argmax(logits, dim=1)
            
            total += y.size(0)
            correct += (predictions == y).sum().item()
            
        acc = 100 * correct / total
        print(f"Testing accuracy: {np.mean(acc)}")

Current Loss:1.6784868240356445
Current Loss:1.6498467922210693
Current Loss:1.6373282670974731
Current Loss:1.5927273035049438
Current Loss:1.5761090517044067
Current Loss:1.546085000038147
Current Loss:1.552997350692749
Current Loss:1.4977425336837769
Current Loss:1.504931926727295
Current Loss:1.4781867265701294
Current Loss:1.4537540674209595
Current Loss:1.4218891859054565
Current Loss:1.423547387123108
Current Loss:1.3836110830307007
Current Loss:1.405530571937561
Current Loss:1.3321576118469238
Current Loss:1.3099793195724487
Current Loss:1.3559396266937256
Current Loss:1.324311375617981
Current Loss:1.3101035356521606
Current Loss:1.242849349975586
Testing accuracy: 70.90671885192432
Current Loss:1.248180627822876
Current Loss:1.2453019618988037
Current Loss:1.1666399240493774
Current Loss:1.1833124160766602
Current Loss:1.1792104244232178
Current Loss:1.176398515701294
Current Loss:1.1343668699264526
Current Loss:1.0790598392486572
Current Loss:1.0903936624526978
Current Loss:

KeyboardInterrupt: 

In [161]:
print("this")

this


In [18]:
total = 0
correct = 0
finetuner.eval()

for x, y in test_loader:
    x = x.to(device)
    y = y.to(device)

#             logits = logreg(x)
    logits = finetuner(x)
    predictions = torch.argmax(logits, dim=1)

    total += y.size(0)
    correct += (predictions == y).sum().item()

acc = 100 * correct / total

print(f"Testing accuracy: {np.mean(acc)}")

RuntimeError: Given input size: (512x4x4). Calculated output size: (512x-2x-2). Output size is too small

In [4]:
torch.save({'model_state_dict':  finetuner.state_dict()}, os.path.join('/home/ubuntu/BYOL/runs/', "finetuned.pth"))

NameError: name 'finetuner' is not defined